<a href="https://colab.research.google.com/github/Saad-Huss41/MovieRex---Movie-Recommendation-System/blob/main/Logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
df1=pd.read_csv('/content/tmdb_5000_credits.csv')
df2=pd.read_csv('/content/tmdb_5000_movies.csv')
# Just reading the files and importing some libraries we'll need.

In [16]:
df1.columns = ['id','title','cast','crew']
df2= df2.merge(df1,on='id')
df2.head(5)
# Concatenate the datasets.

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,tagline,title_x,vote_average,vote_count,title_y,cast_x,crew_x,title,cast_y,crew_y
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [17]:
C= df2['vote_average'].mean()
C

6.092171559442016

In [18]:
m= df2['vote_count'].quantile(0.9)
m

1838.4000000000015

In [19]:
q_movies = df2.copy().loc[df2['vote_count'] >= m]
q_movies.shape
#q_movies are "qualified movies", movies with a certain number (m) of votes.

(481, 26)

In [20]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)
# Calculation based on the IMDB formula


In [21]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [23]:
q_movies = q_movies.sort_values('score', ascending=False)
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)
# Here is our first very basic recommender system.

,title,vote_count,vote_average,score
1881,The Shawshank Redemption,8205,8.5,8.059258
662,Fight Club,9413,8.3,7.939256
65,The Dark Knight,12002,8.2,7.920020
3232,Pulp Fiction,8428,8.3,7.904645
96,Inception,13752,8.1,7.863239
3337,The Godfather,5893,8.4,7.851236
95,Interstellar,10867,8.1,7.809479
809,Forrest Gump,7927,8.2,7.803188
329,The Lord of the Rings: The Return of the King,8064,8.1,7.727243
1990,The Empire Strikes Back,5879,8.2,7.697884


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Tfidf = Term Frequency-Inverse Document Frequency. TfidfVectorizer turns docs into matrices with numerical values showing how important a word is to the doc.

In [25]:
tfidf = TfidfVectorizer(stop_words='english')

In [26]:
df2['overview'] = df2['overview'].fillna('')
# We want to fill any empty descriptions with an empty string.

In [27]:
tfidf_matrix = tfidf.fit_transform(df2['overview'])
# Here we just make the matrix

In [28]:
tfidf_matrix.shape

(4803, 20978)

In [29]:
from sklearn.metrics.pairwise import linear_kernel

In [30]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# The great thing about using linear_kernel here is that it'll perform the same thing as Cosine Similarity except it wont take as long.
# As for what it does, it calculates the dot product of the two vectors.
# Basically it multiplies the two vectors by each other which will tell us how similar each element is to every other element in the Matrix

In [31]:
cosine_sim.shape

(4803, 4803)

In [32]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()


In [33]:
indices[:10]

,0
title,
Avatar,0
Pirates of the Caribbean: At World's End,1
Spectre,2
The Dark Knight Rises,3
John Carter,4
Spider-Man 3,5
Tangled,6
Avengers: Age of Ultron,7
Harry Potter and the Half-Blood Prince,8


In [34]:
# We define a function to return movies with similar scores
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Sort from most similar to least

    sim_scores = sim_scores[1:11]
    # 10 closest scores
    movie_indices = [i[0] for i in sim_scores]

    return df2['title'].iloc[movie_indices]


In [35]:
print(get_recommendations('The Great Gatsby'))

3505               Men of War
3655              Fascination
3056             Cedar Rapids
3033            The Corruptor
637            Les Misérables
3311    Thank You for Smoking
857              Imagine That
4463                 Not Cool
3921    Woman Thou Art Loosed
4074               It Follows
Name: title, dtype: object


In [36]:
# Congratulations!! You have a working recommender system based with *Content Based Filtering* and specifically centered around the plot overview.

In [37]:
from ast import literal_eval
import ast

In [38]:
# Important note for using other datasets: cast_y and crew_x should be changed to cast or crew if thats how the columns are named in the dataset.
# In my case, I named them this way to make this code block run since I had both cast_y, cast_x and crew_y,crew_x in my df2.
features = ['cast_y', 'crew_x', 'keywords', 'genres']
for feature in features:
  df2[feature] = df2[feature].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') and x.endswith(']') else x)
  # We want to validate the list is present before we add it to avoid any errors.

In [39]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan
    # We are getting the director of the movie.

In [40]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [41]:
df2['director'] = df2['crew_x'].apply(get_director)

features = ['cast_y', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(get_list)

In [42]:
df2[['title', 'cast_y', 'director', 'keywords', 'genres']].head(3)

,title,cast_y,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"


In [43]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [44]:
features = ['cast_y', 'keywords', 'director', 'genres']

for feature in features:
    df2[feature] = df2[feature].apply(clean_data)
    # Clean the dataset into lower case and get rid of spaces.

In [45]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast_y']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df2['soup'] = df2.apply(create_soup, axis=1)
# We are making metadata "soup" which is a string with all the data we want to feed our vectorizer.

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])
# Count vectorizer returns how many times each word appears in a text in the form of a matrix.

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
# We are now performing Cosine Similarity which checks how similar two vectors are by measuring the Cosine angle between them.
# Typically used in text document similarity analysis.

In [48]:
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])

In [49]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

,title
65,The Dark Knight
119,Batman Begins
4638,Amidst the Devil's Wings
1196,The Prestige
3073,Romeo Is Bleeding
3326,Black November
1503,Takers
1986,Faster
303,Catwoman
747,Gangster Squad


In [50]:
get_recommendations('The Godfather', cosine_sim2)
# Congratulations again! You have another recommender system based on Keywords, Genres, and Credits.

,title
867,The Godfather: Part III
2731,The Godfather: Part II
4638,Amidst the Devil's Wings
2649,The Son of No One
1525,Apocalypse Now
1018,The Cotton Club
1170,The Talented Mr. Ripley
1209,The Rainmaker
1394,Donnie Brasco
1850,Scarface


In [53]:
!pip install surprise

  Using cached surprise-0.1-py2.py3-none-any.whl.metadata (327 bytes)
  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505187 sha256=0bfe09cfe822b8805a9e1144a440413b6e9a10f2c8dd7d017b5f0fb53c228b5c
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [57]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
reader = Reader()
ratings = pd.read_csv('/content/ratings_small.csv')
ratings.head()
# Reader() reads ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [58]:
reader = Reader(rating_scale=(1, 5))

In [59]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


In [60]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
# A bit to unpack here: SVD() is Single Value Decompisition. We use this to decrease the dimensionality of the matrix by taking the latent factors.
# RMSE is the Root Mean Square Error, a performance indicator for regression models. The lower, the better
# It works by measuring the average difference between values predicted by a model and the actual values. (measures accuracy.)
# MAE = Mean Absolute Error. It calculates the average magnitude of the absolute errors between the predicted and actual values.
# cv is Cross Validaton Iterator. It determines how the data will be split. For further information refer to sklearn's documentation: KFold — scikit-learn 1.6.dev0 documentation
# verbose=True just tells it to print the accuracy measure for each split.

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9043  0.8968  0.8969  0.8899  0.8997  0.8975  0.0047  
MAE (testset)     0.6981  0.6909  0.6881  0.6851  0.6919  0.6908  0.0043  
Fit time          1.47    1.48    1.44    1.47    1.43    1.46    0.02    
Test time         0.11    0.11    0.13    0.23    0.13    0.14    0.04    


{'test_rmse': array([0.90429313, 0.89676185, 0.8968857 , 0.88991023, 0.89965987]),
 'test_mae': array([0.69806831, 0.69093446, 0.68806969, 0.68507555, 0.69186606]),
 'fit_time': (1.4699130058288574,
  1.4797422885894775,
  1.44279146194458,
  1.4677410125732422,
  1.4274840354919434),
 'test_time': (0.11345839500427246,
  0.10646724700927734,
  0.13253402709960938,
  0.2286977767944336,
  0.1342306137084961)}

In [61]:
trainset = data.build_full_trainset()
svd.fit(trainset)
# We build our training set from which we can query predictions.
# .fit() actually did the training

In [62]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [63]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.871767973213684, details={'was_impossible': False})

In [64]:
# For one final time, congratulations!!! I hope this helped you out and was not too difficult of a read. I tried to make this tutorial easy for beginners to understand, but I think any confusion can be solved with reading the documentation.
# Thank you for reading!

In [65]:
import sqlite3
conn = sqlite3.connect('movierex.db')

In [66]:
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    preferences TEXT,
    top5_movies TEXT
)
''')

conn.commit()

In [67]:
def add_user(name, email, preferences, top5_movies):
    try:
        cursor.execute('''
        INSERT INTO users (name, email, preferences, top5_movies)
        VALUES (?, ?, ?, ?)
        ''', (name, email, preferences, top5_movies))
        conn.commit()
        print(f"User {name} added successfully.")
    except sqlite3.IntegrityError as e:
        print(f"Error: {e}")


In [81]:
def generate_recommendations(user_id, cosine_sim):
    cursor.execute('SELECT top5_movies FROM users WHERE user_id = ?', (user_id,))
    result = cursor.fetchone()

    if not result:
        print("User not found.")
        return []

    top5_movies = result[0].split(', ')

    recommendations = set()
    for movie in top5_movies:
      if movie and movie in indices:
        recommendations.update(get_recommendations(movie, cosine_sim))  # Pass cosine_sim

    recommendations.difference_update(top5_movies)

    return list(recommendations)[:5]

In [82]:
if __name__ == "__main__":
    name = input("Enter your name: ")
    email = input("Enter your email: ")
    preferences = input("Enter your movie preferences (comma-separated genres): ")

    print("Enter your top 5 favorite movies (one per line):")
    top5_movies = [input(f"Movie {i+1}: ") for i in range(5)]
    top5_movies_str = ", ".join(top5_movies)

    add_user(name, email, preferences, top5_movies_str)

    user_id = cursor.lastrowid  # Get the last inserted user ID
    recs = generate_recommendations(user_id, cosine_sim)

    print(f"\nTop 5 movie recommendations for you: {recs}")

conn.close()

Enter your name: bob3
Enter your email: bob3@gmail.com
Enter your movie preferences (comma-separated genres): thriller, horror, sci-fi, action, adventure
Enter your top 5 favorite movies (one per line):
Movie 1: The Dark Knight
Movie 2: The Dark Knight Rises
Movie 3: Spider-Man 3
Movie 4: Avatar
Movie 5: Tangled
User bob3 added successfully.

Top 5 movie recommendations for you: ['Raising Victor Vargas', 'Your Highness', 'An American in Hollywood', 'The Thief and the Cobbler', 'Batman v Superman: Dawn of Justice']
